<a href="https://colab.research.google.com/github/PajrulPalah/project_internship_indekstat/blob/main/Analisa_Data_Magang_Menggunakan_Query_SQL_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Modul Untuk Menganlisis Data Menggunakan Query di Postgree SQL**

### Created  by: Muhammad Pajrul Palah
### Selasa, 11 November 2023

Sebelum mengcopy script di bawah ini sebagai reminder dalam running query harus terkoneksasi antara dbeaver dan postgree kemudian dataset data_magang_dprd_kab harus sudah tersimpan di local database aplikasi software tersebut.

Dan ketika running query harus dilakukan satu persatu yang diakiri tanda (;) dengan cara memblok setiap satu query kemudian menjalanknya.

Harap Perhatikan Setiap Komentar Yang Diawali (- -) Sebagai Tajuk Pembahasahan Terkait Hasil Query Yang Akan Dimunculkan

##**Silakan Untuk Dicopy Query SQL Berikut**

```(r)
-- MENGANALISA TERKAIT DATA BERDSARKAN DAPIL MASING-MASING DI SETIAP WILAYAH YANG MEMILIKI POTENSI PEROLEHAN SUARA DIATAS ATAU SAMA DENGAN 20 PERSEN
-- Memeriksa total dapil yang dalam dataset
select distinct nm_dapil
from public.data_magang_dprd_kab
order by nm_dapil asc;

-- Untuk Mengetahui total dapil dalam dataset tersebut (seluruh Indonesia)
select count(distinct nm_dapil) as total_dapil_indonesia_2019
from public.data_magang_dprd_kab;

-- Melakukan analisa total perolehan suara dan persentasenya berdasarkan dapil masing-masing
with RankedPartai as (
    select
        subquery.nm_dapil,  
        subquery.nm_kota,
        subquery.nm_prov,
        subquery.mask_partai,
        subquery.total_perolehan_suara,
        (subquery.total_perolehan_suara * 100.0) / total.total_perolehan_dapil as persentase_keterlibatan_partai,
        rank() over (partition by subquery.nm_dapil
        order by
            subquery.total_perolehan_suara desc) as rank_partai
    from
        (
        select
            nm_dapil,  
            nm_kota,
            nm_prov,
            mask_partai,
            SUM(perolehan) as total_perolehan_suara
        from
            data_magang_dprd_kab
        group by
            nm_dapil,  
            nm_kota,
            nm_prov,
            mask_partai
        ) subquery
    join
        (
        select
            nm_dapil,
            SUM(perolehan) as total_perolehan_dapil
        from
            data_magang_dprd_kab
        group by
            nm_dapil
        ) total
    on
        subquery.nm_dapil = total.nm_dapil
)
select
    *
from
    RankedPartai
order by
    nm_dapil,  
    nm_kota,
    rank_partai;

-- Memfilter hanya partai yang memiliki nilai persentase di atas atau sama dengan 20 % perdapil masing-masing

with RankedPartai as (
    select
        subquery.nm_dapil,  
        subquery.nm_kota,
        subquery.nm_prov,
        subquery.mask_partai,
        subquery.perolehan_suara_perdapil,
        (subquery.perolehan_suara_perdapil * 100.0) / total.total_perolehan_dapil as persentase_keterlibatan_partai,
        rank() over (partition by subquery.nm_dapil
        order by
            subquery.perolehan_suara_perdapil desc) as rank_partai
    from
        (
        select
            nm_dapil,  
            nm_kota,
            nm_prov,
            mask_partai,
            SUM(perolehan) as perolehan_suara_perdapil
        from
            data_magang_dprd_kab
        group by
            nm_dapil,  
            nm_kota,
            nm_prov,
            mask_partai
        ) subquery
    join
        (
        select
            nm_dapil,
            SUM(perolehan) as total_perolehan_dapil
        from
            data_magang_dprd_kab
        group by
            nm_dapil
        ) total
    on
        subquery.nm_dapil = total.nm_dapil
)
select
    *
from
    RankedPartai
where
    persentase_keterlibatan_partai >= 20
order by
    nm_dapil,  
    nm_kota,
    rank_partai;

-- Untuk Mengetahui total rows yang ada pada hasil query di atas
with RankedPartai as (
    select
        subquery.nm_dapil,  
        subquery.nm_kota,
        subquery.nm_prov,
        subquery.mask_partai,
        subquery.perolehan_suara_perdapil,
        (subquery.perolehan_suara_perdapil * 100.0) / total.total_perolehan_dapil as persentase_keterlibatan_partai,
        rank() over (partition by subquery.nm_dapil
        order by
            subquery.perolehan_suara_perdapil desc) as rank_partai
    from
        (
        select
            nm_dapil,  
            nm_kota,
            nm_prov,
            mask_partai,
            SUM(perolehan) as perolehan_suara_perdapil
        from
            data_magang_dprd_kab
        group by
            nm_dapil,  
            nm_kota,
            nm_prov,
            mask_partai
        ) subquery
    join
        (
        select
            nm_dapil,
            SUM(perolehan) as total_perolehan_dapil
        from
            data_magang_dprd_kab
        group by
            nm_dapil
        ) total
    on
        subquery.nm_dapil = total.nm_dapil
)

select count(*) as total_rows
from RankedPartai
where persentase_keterlibatan_partai >= 20;


-- Analisa lebih lanjut terkait suara yang lebih dari 20% dengan menambahkan kolom total desa dan total kecamatan, agar mengetahui dan dapat dianalisa lebih lanjut
with RankedPartai as (
select
	subquery.nm_dapil,
	subquery.nm_kota,
	subquery.nm_prov,
	subquery.mask_partai,
	subquery.perolehan_suara_perdapil,
	(subquery.perolehan_suara_perdapil * 100.0) / total.total_perolehan_dapil as persentase_keterlibatan_partai,
	rank() over (partition by subquery.nm_dapil
order by
	subquery.perolehan_suara_perdapil desc) as rank_partai
from
	(
	select
		nm_dapil,
		nm_kota,
		nm_prov,
		mask_partai,
		SUM(perolehan) as perolehan_suara_perdapil
	from
		data_magang_dprd_kab
	group by
		nm_dapil,
		nm_kota,
		nm_prov,
		mask_partai
    ) subquery
join (
	select
		nm_dapil,
		SUM(perolehan) as total_perolehan_dapil
	from
		data_magang_dprd_kab
	group by
		nm_dapil
    ) total on
	subquery.nm_dapil = total.nm_dapil
),
TotalKecamatanDesa as (
select
	nm_dapil,
	COUNT(distinct nm_kec) as total_kecamatan,
	COUNT(distinct nm_desa) as total_desa
from
	data_magang_dprd_kab
group by
	nm_dapil
)
select
	RankedPartai.*,
	tkd.total_kecamatan,
	tkd.total_desa
from
	RankedPartai
join TotalKecamatanDesa tkd on
	RankedPartai.nm_dapil = tkd.nm_dapil
where
	persentase_keterlibatan_partai >= 20
order by
	nm_dapil,
	nm_kota,
	rank_partai;

-- Untuk Mengkonfirmasi apakah query berjalan dan sesuai dengan hasil yang dimunculkan
select count(nm_desa) as total_desa, count ( distinct nm_kec) as  total_kec
from data_magang_dprd_kab
where nm_dapil = 'ACEH BARAT 2';
-- Dari hasil query diatas terdapat kejanggalan dalam total desa sebanyak 1520 desa di 3 kecamatan

-- Untuk Mengkonfirmasi apakah query berjalan dan sesuai dengan hasil yang dimunculkan
select count(distinct nm_desa) as total_desa, count( distinct nm_kec) as  total_kec
from data_magang_dprd_kab
where nm_dapil = 'ACEH BARAT 2';
-- Dalam Hasil Query ini memungkinkan ada nama desa yang sama di 3 kecamatan yang namanya sama.

-- Untuk Memunculkan desa dan kecamatan
with RankedPartai as (
    select
        subquery.nm_dapil,
        subquery.nm_kota,
        subquery.nm_prov,
        subquery.mask_partai,
        subquery.perolehan_suara_perdapil,
        (subquery.perolehan_suara_perdapil * 100.0) / total.total_perolehan_dapil as persentase_keterlibatan_partai,
        rank() over (partition by subquery.nm_dapil
        order by
        subquery.perolehan_suara_perdapil desc) as rank_partai
    from
        (
        select
            nm_dapil,
            nm_kota,
            nm_prov,
            mask_partai,
            SUM(perolehan) as perolehan_suara_perdapil
        from
            data_magang_dprd_kab
        group by
            nm_dapil,
            nm_kota,
            nm_prov,
            mask_partai
        ) subquery
    join (
        select
            nm_dapil,
            SUM(perolehan) as total_perolehan_dapil
        from
            data_magang_dprd_kab
        group by
            nm_dapil
        ) total on
        subquery.nm_dapil = total.nm_dapil
),
TotalKecamatanDesa as (
    select
        nm_dapil,
        COUNT(distinct nm_kec) as total_kecamatan,
        COUNT(distinct nm_desa) as total_desa
    from
        data_magang_dprd_kab
    group by
        nm_dapil
)
select
    RankedPartai.*,
    tkd.total_kecamatan,
    tkd.total_desa,
    data_magang_dprd_kab.nm_desa,
    data_magang_dprd_kab.nm_kec,
    data_magang_dprd_kab.perolehan
from
    RankedPartai
join TotalKecamatanDesa tkd on
    RankedPartai.nm_dapil = tkd.nm_dapil
join data_magang_dprd_kab on
    RankedPartai.nm_dapil = data_magang_dprd_kab.nm_dapil
    and RankedPartai.mask_partai = data_magang_dprd_kab.mask_partai
where
    persentase_keterlibatan_partai >= 20
order by
    nm_dapil,
    nm_kota,
    rank_partai;

 -- Memunculkan data hanya provinsi tertentu dengan masing-masing nama desa dan kecamatan
with RankedPartai as (
select
	subquery.nm_dapil,
	subquery.nm_kota,
	subquery.nm_prov,
	subquery.mask_partai,
	subquery.perolehan_suara_perdapil,
	(subquery.perolehan_suara_perdapil * 100.0) / total.total_perolehan_dapil as persentase_keterlibatan_partai,
	rank() over (partition by subquery.nm_dapil
order by
	subquery.perolehan_suara_perdapil desc) as rank_partai
from
	(
	select
		nm_dapil,
		nm_kota,
		nm_prov,
		mask_partai,
		SUM(perolehan) as perolehan_suara_perdapil
	from
		data_magang_dprd_kab
	group by
		nm_dapil,
		nm_kota,
		nm_prov,
		mask_partai
        ) subquery
join (
	select
		nm_dapil,
		SUM(perolehan) as total_perolehan_dapil
	from
		data_magang_dprd_kab
	group by
		nm_dapil
        ) total on
	subquery.nm_dapil = total.nm_dapil
),
TotalKecamatanDesa as (
select
	nm_dapil,
	COUNT(distinct nm_kec) as total_kecamatan,
	COUNT(distinct nm_desa) as total_desa
from
	data_magang_dprd_kab
group by
	nm_dapil
)
select
	RankedPartai.*,
	tkd.total_kecamatan,
	tkd.total_desa,
	data_magang_dprd_kab.nm_desa,
	data_magang_dprd_kab.nm_kec,
	data_magang_dprd_kab.perolehan
from
	RankedPartai
join TotalKecamatanDesa tkd on
	RankedPartai.nm_dapil = tkd.nm_dapil
join data_magang_dprd_kab on
	RankedPartai.nm_dapil = data_magang_dprd_kab.nm_dapil
	and RankedPartai.mask_partai = data_magang_dprd_kab.mask_partai
where
	persentase_keterlibatan_partai >= 20
	and data_magang_dprd_kab.nm_prov in (
        'KALIMANTAN UTARA',
        'SULAWESI TENGAH',
        'SULAWESI SELATAN',
        'SULAWESI UTARA',
        'SULAWESI TENGGARA',
        'MALUKU UTARA'
    )
order by
	nm_dapil,
	nm_kota,
	rank_partai;

-- SILAKAN UNTUK DIGANTI SESUAI PEMBAGIAN PROVINSI MASING-MASING

-- Untuk Memvalidasi Provinsi yang ada dalam dataset sesuai dengan kondisi where yang diminta antara match atau tidaknya.
 select distinct nm_prov
 from data_magang_dprd_kab dmdk
 where nm_prov in (
        'KALIMANTAN UTARA',
        'SULAWESI TENGAH',
        'SULAWESI SELATAN',
        'SULAWESI UTARA',
        'SULAWESI TENGGARA',
        'MALUKU UTARA');
       
       
       

```